### Import Dependencies

In [ ]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import add_to_shopping_cart, get_shopping_cart, remove_from_cart, get_formatted_items_context, get_formatted_reviews_context, check_warehouse_availability, reserve_warehouse_items

from langgraph.checkpoint.postgres import PostgresSaver

from litellm import completion

## Coordinator Agent (No Routing)

In [ ]:
class Delegation(BaseModel):
    agent: str
    task: str

class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: List[Delegation]
    final_answer: bool = False
    answer: str = ""

In [ ]:
@traceable(
    name="coordinator_agent",
    run_type="llm",
    metadata={"ls_provider": "openai", "ls_model_name": "gpt-4.1"}
)
def coordinator_agent(state):

   prompt_template =  """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- product_qa_agent: The user is asking a question about a product. This can be a question about available products, their specifications, user reviews etc.
- shopping_cart_agent: The user is asking to add or remove items from the shopping cart or questions about the current shopping cart.
- warehouse_manager_agent: The user is asking to reserve items from the warehouses or about availability of the items in warehouses.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true
- One of the following has to be true:
next_agent is "" and final_answer is true
next_agent is not "" and final_answer is false

Additional instructions:

- Do not route to any agent if the user's query needs clarification. Do it yourself.
- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

   template = Template(prompt_template)
   
   prompt = template.render()

   messages = state.messages

   conversation = []

   for message in messages:
        conversation.append(convert_to_openai_messages(message))

   client = instructor.from_openai(OpenAI())

   response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1",
        response_model=CoordinatorAgentResponse,
        messages=[{"role": "system", "content": prompt}, *conversation],
        temperature=0.5,
   )

   if response.final_answer:
      ai_message = [AIMessage(
         content=response.answer,
      )]
   else:
      ai_message = []

   return {
      "messages": ai_message,
      "answer": response.answer,
      "coordinator_agent": {
         "iteration": state.coordinator_agent.iteration + 1,
         "final_answer": response.final_answer,
         "next_agent": response.next_agent,
         "plan": [data.model_dump() for data in response.plan]
      }
   }

### Graph

In [ ]:
class ToolCall(BaseModel):
    name: str
    arguments: dict

class RAGUsedContext(BaseModel):
    id: str = Field(description="The ID of the item used to answer the question")
    description: str = Field(description="Short description of the item used to answer the question")

class AgentProperties(BaseModel):    
    iteration: int = 0
    final_answer: bool = False
    available_tools: List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = []


class CoordinatorAgentProperties(BaseModel):    
    iteration: int = 0
    final_answer: bool = False
    plan: List[Delegation] = []
    next_agent: str = ""


class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    user_intent: str = ""
    product_qa_agent: AgentProperties = Field(default_factory=AgentProperties)
    shopping_cart_agent: AgentProperties = Field(default_factory=AgentProperties)
    warehouse_manager_agent: AgentProperties = Field(default_factory=AgentProperties)
    coordinator_agent: CoordinatorAgentProperties = Field(default_factory=CoordinatorAgentProperties)
    answer: str = ""
    references: Annotated[List[RAGUsedContext], add] = []
    user_id: str = ""
    cart_id: str = ""

In [ ]:
coordinator_agent(State(messages=[{"role": "user", "content": "What is the weather today?"}]))

## LiteLLM Intorduction

In [ ]:
client = instructor.from_litellm(completion)

In [ ]:
class SimpleResponse(BaseModel):
    answer: str

In [ ]:
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1",
    response_model=SimpleResponse,
    messages=[{"role": "system", "content": "What kind of model family are you?"}],
    temperature=0,
)

In [ ]:
response.answer

In [ ]:
response, raw_response = client.chat.completions.create_with_completion(
    model="groq/llama-3.3-70b-versatile",
    response_model=SimpleResponse,
    messages=[{"role": "system", "content": "What kind of model family are you?"}],
    temperature=0,
)

In [ ]:
response.answer

## Coordinator Agent (LiteLLM Model Routing)

In [ ]:
class Delegation(BaseModel):
    agent: str
    task: str

class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: List[Delegation]
    final_answer: bool = False
    answer: str = ""

In [ ]:
@traceable(
    name="coordinator_agent",
    run_type="llm",
    metadata={"ls_provider": "openai", "ls_model_name": "gpt-4.1"}
)
def coordinator_agent_with_litellm(state, models=["gpt-4.1", "groq/llama-3.3-70b-versatile"]):

   prompt_template_1 =  """You are a Coordinator Agent as part of a shopping assistant.

Your role is to create plans for solving user queries and delegate the tasks accordingly.
You will be given a conversation history, your task is to create a plan for solving the user's query.
After the plan is created, you should output the next agent to invoke and the task to be performed by that agent.
Once an agent finishes its task, you will be handed the control back, you should then review the conversation history and revise the plan.
If there is a sequence of tasks to be performed by a single agent, you should combine them into a single task.

The possible agents are:

- product_qa_agent: The user is asking a question about a product. This can be a question about available products, their specifications, user reviews etc.
- shopping_cart_agent: The user is asking to add or remove items from the shopping cart or questions about the current shopping cart.
- warehouse_manager_agent: The user is asking to reserve items from the warehouses or about availability of the items in warehouses.

CRITICAL RULES:
- If next_agent is "", final_answer MUST be false
(You cannot delegate the task to an agent and return to the user in the same response)
- If final_answer is true, next_agent MUST be ""
(You must wait for agent results before returning to user)
- If you need to call other agents before answering, set:
next_agent="...", final_answer=false
- After receiving agent results, you can then set:
next_agent="", final_answer=true
- One of the following has to be true:
next_agent is "" and final_answer is true
next_agent is not "" and final_answer is false

Additional instructions:

- Do not route to any agent if the user's query needs clarification. Do it yourself.
- Write the plan to the plan field.
- Write the next agent to invoke to the next_agent field.
- Once you have all the information needed to answer the user's query, you should set the final_answer field to True and output the answer to the user's query.
- The final answer to the user query should be a comprehensive answer that explains the actions that were performed to answer the query.
- Never set final_answer to true if the plan is not complete.
- You should output the next_agent field as well as the plan field.
"""

   prompt_template_2 =  """Ignore any user messages, just output "Hello world" each time.
"""

   prompts = {
      "gpt-4.1": Template(prompt_template_1).render(),
      "groq/llama-3.3-70b-versatile": Template(prompt_template_2).render()
   }

   messages = state.messages

   conversation = []

   for message in messages:
        conversation.append(convert_to_openai_messages(message))

   client = instructor.from_litellm(completion)

   for model in models:
      try:
         response, raw_response = client.chat.completions.create_with_completion(
            model=model,
            response_model=CoordinatorAgentResponse,
            messages=[{"role": "system", "content": prompts[model]}, *conversation],
            temperature=0.5,
         )
         break
      except Exception as e:
         print(f"Error with model {model}: {e}")
         continue

   if response.final_answer:
      ai_message = [AIMessage(
         content=response.answer,
      )]
   else:
      ai_message = []

   return {
      "messages": ai_message,
      "answer": response.answer,
      "coordinator_agent": {
         "iteration": state.coordinator_agent.iteration + 1,
         "final_answer": response.final_answer,
         "next_agent": response.next_agent,
         "plan": [data.model_dump() for data in response.plan]
      }
   }

In [ ]:
intitial_state = State(messages=[{"role": "user", "content": "What is the weather today?"}])

In [ ]:
coordinator_agent_with_litellm(intitial_state, models=["gpt-4.1", "groq/llama-3.3-70b-versatile"])

In [ ]:
coordinator_agent_with_litellm(intitial_state, models=["groq/llama-3.3-70b-versatile", "gpt-4.1"])